In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import joblib


df = pd.read_csv(r"D:\game\climate_forecast_project\data\co2_monthly.csv", parse_dates=["date"])
target_col = "value"


scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[[target_col]])


N = 12 
X, y = [], []
for i in range(len(scaled) - N):
    X.append(scaled[i:i+N])  
    y.append(scaled[i+N])

X = np.array(X)  
y = np.array(y) 

# --- Convert to tensors ---
X_tensor = torch.from_numpy(X).float()          
y_tensor = torch.from_numpy(y).float()        

print("X_tensor shape:", X_tensor.shape)
print("y_tensor shape:", y_tensor.shape)


class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
       
        out, _ = self.lstm(x)
        out = out[:, -1, :]  
        out = self.fc(out)
        return out

model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)    
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")


torch.save(model.state_dict(), r"D:\game\climate_forecast_project\models\lstm_model.pt")
joblib.dump(scaler, r"D:\game\climate_forecast_project\models\scaler.pkl")


model.eval()
last_seq = torch.from_numpy(scaled[-N:].reshape(1, N, 1)).float()
with torch.no_grad():
    pred_scaled = model(last_seq).numpy()
pred = scaler.inverse_transform(pred_scaled.reshape(-1,1))
print("Next month forecast:", pred.flatten())


X_tensor shape: torch.Size([288, 12, 1])
y_tensor shape: torch.Size([288, 1])
Epoch 10/50, Loss: 0.246536
Epoch 20/50, Loss: 0.046171
Epoch 30/50, Loss: 0.041642
Epoch 40/50, Loss: 0.036640
Epoch 50/50, Loss: 0.026784
Next month forecast: [414.84372]
